# ***Abstract :***
# This project is built to detect the depressive based on the tweet content user posted. 

***Recommend using colab to run my program***

In [1]:
# pip and import necessary file and library

! git clone https://github.com/Jingxuan-Bao/Tweet_Depression_Detection.git
! pip install contractions
! pip install joblib
! pip install files
! pip install transformers

import pandas as pd
import re
import string
import nltk
import contractions
import numpy as np
nltk.download('stopwords')


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from google.colab import files
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score

import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

fatal: destination path 'Tweet_Depression_Detection' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Section 1: Load Data**

In [2]:
# read normal tweet data from my github

normal_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/normal_tweet.csv')
print(normal_tweet.head())

                                                text  label
0  in class. im EARLY. bummed RYAN SEACREST never...      0
1                                   i wanna go home       0
2                                  Our AC is broken       0
3  feels that the only place where Federer (&amp;...      0
4  @tommcfly Yeah tom, you are always the one tha...      0


In [3]:
normal_tweet

,text,label
0,in class. im EARLY. bummed RYAN SEACREST never...,0
1,i wanna go home,0
2,Our AC is broken,0
3,feels that the only place where Federer (&amp;...,0
4,"@tommcfly Yeah tom, you are always the one tha...",0
...,...,...
59995,It's 8:40am... I've been awake for nearly 2 ho...,0
59996,ughh mom`s making me cook dinner when i`m in t...,0
59997,"argh. netregistry is pissing me off today, pos...",0
59998,@gemmak500 It's amazing how many couples you s...,0


In [4]:
# read depressive tweet data from my github

depressive_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/depressive_tweets.csv')
print(depressive_tweet.head())

   Unnamed: 0             tweet.id           created_at  \
0           0  1447537898572574730  2021-10-11 12:21:43   
1           1  1447540582490988553  2021-10-11 12:32:23   
2           2  1447807717859491842  2021-10-12 06:13:53   
3           3  1448076026219692033  2021-10-13 00:00:03   
4           4  1448382047375040513  2021-10-13 20:16:04   

                                                text          location  \
0  Open discussion. Between the Transfer Portal a...  Cheyenne Wyoming   
1  Plenty of things are changing in my life and t...               NaN   
2  I feel a little hopeless. Anyone else? #hopele...               NaN   
3  Which is more healthy? Hope, or hopelessness? ...        Denver, CO   
4  So someone tell me how do I get over #HOPELESS...    Portland Or .    

   retweet  favorite  
0        0         0  
1        0         0  
2        0         0  
3        0         0  
4        0         2  


In [5]:
depressive_tweet

,Unnamed: 0,tweet.id,created_at,text,location,retweet,favorite
0,0,1447537898572574730,2021-10-11 12:21:43,Open discussion. Between the Transfer Portal a...,Cheyenne Wyoming,0,0
1,1,1447540582490988553,2021-10-11 12:32:23,Plenty of things are changing in my life and t...,NaN,0,0
2,2,1447807717859491842,2021-10-12 06:13:53,I feel a little hopeless. Anyone else? #hopele...,NaN,0,0
3,3,1448076026219692033,2021-10-13 00:00:03,"Which is more healthy? Hope, or hopelessness? ...","Denver, CO",0,0
4,4,1448382047375040513,2021-10-13 20:16:04,So someone tell me how do I get over #HOPELESS...,Portland Or .,0,2
...,...,...,...,...,...,...,...
24142,24142,1459521498842992642,2021-11-13 14:00:16,Just got banned from a server F #sad,Jakarta Capital Region,0,1
24143,24143,1459521611997003777,2021-11-13 14:00:43,I literally cried during my exam and the cam i...,بيت أمك,0,0
24144,24144,1459524263946326017,2021-11-13 14:11:15,No one can be happy with a guy like me. That's...,"Varanasi, Uttar Pradesh, India",0,0
24145,24145,1459530315437785095,2021-11-13 14:35:18,arrived at my house but Am I Home? #deep #sad ...,they19sea,1,3


In [6]:
# preprocess the depressive tweet, extract the target content, clean null data

depressive_tweet['label'] = 1
depressive_tweet = depressive_tweet[['text', 'label']]
depressive_tweet = depressive_tweet.dropna()

In [7]:
depressive_tweet

,text,label
0,Open discussion. Between the Transfer Portal a...,1
1,Plenty of things are changing in my life and t...,1
2,I feel a little hopeless. Anyone else? #hopele...,1
3,"Which is more healthy? Hope, or hopelessness? ...",1
4,So someone tell me how do I get over #HOPELESS...,1
...,...,...
24142,Just got banned from a server F #sad,1
24143,I literally cried during my exam and the cam i...,1
24144,No one can be happy with a guy like me. That's...,1
24145,arrived at my house but Am I Home? #deep #sad ...,1


# **Section 2: Data Processing**

In [8]:
# merge normal tweet and depressive tweet

df = pd.concat([normal_tweet,depressive_tweet])
# shuffle the data
df = df.sample(frac=1)

In [9]:
df

,text,label
15493,@JosephDexter i wish i coulda made the drive!...,0
45107,@alissandra hope so! we need to at least hug ...,0
4686,"@jennnnnnx never mind, mother's day oh well w...",0
17921,Crazy how the things we have anxiety about the...,1
42204,(via @etanowitz) There is an @orlandotweetup p...,0
...,...,...
6692,I'm really scared to check myself into a hospi...,1
16503,Bipolar Daily Goals \n❤Take meds 💊\n🧡Cut chick...,1
8893,Around 17 lives are lost each day in England t...,1
8668,@figgybean yeah it concerned me a little cause...,0


In [10]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_tweet_text(tweet_text):
    # Conver to string 
    tweet_text = str(tweet_text)

    # Convert to lowercase
    tweet_text = tweet_text.lower()

    # Expand contractions
    tweet_text = contractions.fix(tweet_text)

    # Remove URLs
    tweet_text = re.sub(r"http\S+", "", tweet_text)

    # Remove mentions (@) and hashtags (#)
    tweet_text = re.sub(r'\@\w+|\#', '', tweet_text)

    # Remove emojis
    tweet_text = remove_emojis(tweet_text)

    # Remove punctuation
    tweet_text = tweet_text.translate(str.maketrans("", "", string.punctuation))

    # Remove numbers
    tweet_text = re.sub(r"\d+", "", tweet_text)

    # Remove whitespace
    tweet_text = tweet_text.strip()

    # Tokenize the text into individual words
    tweet_words = tweet_text.split()

    # Remove stop words using NLTK library
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tweet_words = [word for word in tweet_words if word not in stop_words]

    # Join the remaining words back into a string
    processed_tweet_text = " ".join(tweet_words)

    return processed_tweet_text

In [11]:
df['text'] = df['text'].apply(preprocess_tweet_text)

In [12]:
df

,text,label
15493,wish coulda made drive tell yo thurz said what...,0
45107,hope need least hug goodbye forever point,0
4686,never mind mothers day oh well figure something,0
17921,crazy things anxiety wind really worth worryin...,1
42204,via photo gallery amkfh otweet oh yeah pic pho...,0
...,...,...
6692,really scared check hospital even scared menta...,1
16503,bipolar daily goals ❤take meds 🧡cut chicken tv...,1
8893,around lives lost day england suicide set ➡️ m...,1
8668,yeah concerned little say seems annoyed got go...,0


In [13]:
df = df.sample(n=5000, random_state=42)

# **Section 3: Pre Trained Model - BERT**

In [14]:
# Load dataset
data = df
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Set up tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize data
train_encodings = tokenizer(train_data['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_data['text'].tolist(), truncation=True, padding=True, max_length=128)

# Create custom Dataset class
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_data['label'].tolist())
val_dataset = TweetDataset(val_encodings, val_data['label'].tolist())

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    evaluation_strategy="steps"
)

# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train model
trainer.train()
eval_results = trainer.evaluate()

# Generate classification repoty
val_predictions = trainer.predict(val_dataset).label_ids
val_labels = val_data['label'].tolist()
print(classification_report(val_labels, val_predictions))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss
100,0.204500,0.117322
200,0.098000,0.062771
300,0.074500,0.098976
400,0.045500,0.061898
500,0.041800,0.044210
600,0.021900,0.051490


Step,Training Loss,Validation Loss
100,0.204500,0.117322
200,0.098000,0.062771
300,0.074500,0.098976
400,0.045500,0.061898
500,0.041800,0.044210
600,0.021900,0.051490
700,0.016100,0.080063


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       726
           1       1.00      1.00      1.00       274

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

